# Lab 6.1 - MNIST MLP with PyTorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import GridSearchCV
from skorch import NeuralNetClassifier

In [ ]:
torch.random.manual_seed(0)

In [ ]:
# Define the neural network
class MLP(nn.Module):
    def __init__(self, num_layers=10, num_units=128, dropout=0.5):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.input_layer = nn.Linear(28*28, num_units)
        self.relu = nn.ReLU()
        self.hidden_layers = nn.ModuleList(
            [nn.Linear(num_units, num_units) for _ in range(num_layers)])
        self.output_layer = nn.Linear(num_units, 10)
        self.dropout = nn.Dropout(dropout)


    def forward(self, x):
        x = self.flatten(x)
        x = self.input_layer(x)
        x = self.relu(x)
        for hidden_layer in self.hidden_layers:
            x = hidden_layer(x)
            x = self.relu(x)
            x = self.dropout(x)
        x = self.output_layer(x)
        return x

In [ ]:
# Load the MNIST dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(
    root="./data", train=True, download=True, transform=transform)

testset = torchvision.datasets.MNIST(
    root="./data", train=False, download=True, transform=transform)

In [ ]:
# Convert trainset and testset to numpy arrays
X_train = trainset.data.numpy()
y_train = trainset.targets.numpy()
X_test = testset.data.numpy()
y_test = testset.targets.numpy()

# Convert numpy arrays to tensors
X_train = torch.from_numpy(X_train).to(torch.float)
y_train = torch.from_numpy(y_train)
X_test = torch.from_numpy(X_test).to(torch.float)
y_test = torch.from_numpy(y_test)

print("X_train.shape", X_train.shape)
print("y_train.shape", y_train.shape)
print("X_train.dtype", X_train.dtype)
print("y_train.dtype", y_train.dtype)

X_train.shape torch.Size([60000, 28, 28])
y_train.shape torch.Size([60000])
X_train.dtype torch.float32
y_train.dtype torch.int64


In [ ]:
print("Labels", set(y_train.numpy()))

Labels {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [ ]:
# Wrap the MLP into a scikit-learn classifier
mlp = MLP()
# mlp.to(torch.device("cuda"))
model = NeuralNetClassifier(
    mlp,
    max_epochs=50,
    lr=0.001,
    optimizer=optim.Adam,
    criterion=nn.CrossEntropyLoss,
    verbose=True,
)


In [ ]:
# Define hyperparameters for grid search
params = {
    "lr": [0.0001, 0.001],
    "module__num_layers": [3, 5],
    "module__num_units": [128, 256],
    "max_epochs": [50],
}

grid_search = GridSearchCV(estimator=model, param_grid=params, cv=3, verbose=2)
grid_search.fit(X_train, y_train)


Fitting 3 folds for each of 8 candidates, totalling 24 fits
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.4395       0.6560        1.4228  2.1830
      2        1.4204       0.8241        0.6592  1.8642
      3        0.9120       0.8819        0.4254  1.8654
      4        0.6820       0.9042        0.3386  1.9449
      5        0.5499       0.9181        0.2905  1.9649
      6        0.4588       0.9273        0.2652  2.5458
      7        0.3903       0.9323        0.2465  1.9108
      8        0.3475       0.9387        0.2281  1.9108
      9        0.3149       0.9431        0.2190  2.8152
     10        0.2740       0.9446        0.2139  2.1997
     11        0.2501       0.9496        0.1989  3.4795
     12        0.2279       0.9509        0.1932  2.4945
     13        0.2102       0.9511        0.1901  1.9170
     14        0.1943       0.9535        0.1882  1.8958
     15        0.1763       

GridSearchCV(cv=3,
             estimator=<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=MLP(
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (input_layer): Linear(in_features=784, out_features=128, bias=True)
    (relu): ReLU()
    (hidden_layers): ModuleList(
      (0-9): 10 x Linear(in_features=128, out_features=128, bias=True)
    )
    (output_layer): Linear(in_features=128, out_features=10, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  ),
),
             param_grid={'lr': [0.0001, 0.001], 'max_epochs': [50],
                         'module__num_layers': [3, 5],
                         'module__num_units': [128, 256]},
             verbose=2)

In [ ]:
best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)

Best hyperparameters: {'lr': 0.0001, 'max_epochs': 50, 'module__num_layers': 5, 'module__num_units': 256}


In [ ]:
# Find the model with the best parameters
best_model = grid_search.best_estimator_

In [ ]:
best_model.fit(X_train, y_train)

Re-initializing module because the following parameters were re-set: num_layers, num_units.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.1124       0.5838        1.1552  6.2696
      2        0.9383       0.8898        0.4071  6.6761
      3        0.5178       0.9348        0.2550  6.1313
      4        0.3504       0.9487        0.2122  6.8096
      5        0.2635       0.9557        0.1862  6.2095
      6        0.2176       0.9582        0.1775  6.8460
      7        0.1914       0.9624        0.1616  6.2218
      8        0.1663       0.9634        0.1505  6.8586
      9        0.1438       0.9651        0.1611  6.2918
     10        0.1261       0.9683        0.1447  6.8320
     11        0.1140       0.9680        0.1454  6.8983
     12        0.1040       0.9699        0.1392  6.2754
     13        0.0943       0.9696        0.1500  6.9396

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MLP(
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (input_layer): Linear(in_features=784, out_features=256, bias=True)
    (relu): ReLU()
    (hidden_layers): ModuleList(
      (0-4): 5 x Linear(in_features=256, out_features=256, bias=True)
    )
    (output_layer): Linear(in_features=256, out_features=10, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  ),
)

In [ ]:
best_model.score(X_test, y_test)


0.9788

In [ ]:
# TODO: Load another dataset of your choice.

In [ ]:
# TODO: Reimplement the MLP class to work with the new dataset.

In [ ]:
# TODO: Wrap the MLP into a scikit-learn classifier

In [ ]:
# TODO: Perform grid search to find the best hyperparameters.

In [ ]:
# TODO: Train and evaluate the MLP on the new dataset using the best hyperparameters.